In [2]:
import pandas as pd
import numpy as np
import os

# Session 特徵

### session_length：該 session 的事件總數

In [3]:
def session_length(df_candidates, df_events):
    session_ids, session_counts = np.unique(df_events["session"].values, return_counts=True)
    session_length_map = dict(zip(session_ids, session_counts))  # 轉成字典

    # 使用 Pandas `.map()` 來加速查找，避免回圈
    return df_candidates["session"].map(session_length_map).fillna(0).astype(np.int32).values


### session_click_ratio：點擊事件數 / 總事件數

In [4]:
def session_click_ratio(df_candidates, df_events):
    # 計算每個 session 的總事件數
    session_ids, session_counts = np.unique(df_events["session"].values, return_counts=True)
    session_length_map = dict(zip(session_ids, session_counts))  

    # 計算每個 session 的點擊事件數
    click_events = df_events[df_events["type"] == "clicks"]
    click_session_ids, click_counts = np.unique(click_events["session"].values, return_counts=True)
    session_click_map = dict(zip(click_session_ids, click_counts))  

    # 透過 `.map()` 查找 session 的總事件數與點擊事件數
    total_events = df_candidates["session"].map(session_length_map).fillna(0)
    click_events = df_candidates["session"].map(session_click_map).fillna(0)

    # 計算點擊比例，避免除以 0
    click_ratio = np.where(total_events > 0, click_events / total_events, 0)
    
    # 四捨五入到小數點第四位
    return np.round(click_ratio, 4)

### session_cart_ratio：加入購物車事件數 / 總事件數

In [5]:
def session_cart_ratio(df_candidates, df_events):
    # 計算每個 session 的總事件數
    session_ids, session_counts = np.unique(df_events["session"].values, return_counts=True)
    session_length_map = dict(zip(session_ids, session_counts))  

    # 計算每個 session 的「加入購物車」事件數
    cart_events = df_events[df_events["type"] == "carts"]
    cart_session_ids, cart_counts = np.unique(cart_events["session"].values, return_counts=True)
    session_cart_map = dict(zip(cart_session_ids, cart_counts))  

    # 透過 `.map()` 查找 session 的總事件數與加入購物車事件數
    total_events = df_candidates["session"].map(session_length_map).fillna(0)
    cart_events = df_candidates["session"].map(session_cart_map).fillna(0)

    # 計算加入購物車比例，避免除以 0
    cart_ratio = np.where(total_events > 0, cart_events / total_events, 0)
    
    # 四捨五入到小數點第四位
    return np.round(cart_ratio, 4)

### session_order_ratio：購買事件數 / 總事件數

In [6]:
def session_order_ratio(df_candidates, df_events):
    # 計算每個 session 的總事件數
    session_ids, session_counts = np.unique(df_events["session"].values, return_counts=True)
    session_length_map = dict(zip(session_ids, session_counts))  

    # 計算每個 session 的「購買」事件數
    buy_events = df_events[df_events["type"] == "orders"]
    buy_session_ids, buy_counts = np.unique(buy_events["session"].values, return_counts=True)
    session_buy_map = dict(zip(buy_session_ids, buy_counts))  

    # 透過 `.map()` 查找 session 的總事件數與購買事件數
    total_events = df_candidates["session"].map(session_length_map).fillna(0)
    buy_events = df_candidates["session"].map(session_buy_map).fillna(0)

    # 計算購買比例，避免除以 0
    order_ratio = np.where(total_events > 0, buy_events / total_events, 0)
    
    # 四捨五入到小數點第四位
    return np.round(order_ratio, 4)

### session_last_click_aid：最後一次點擊的商品 ID

In [7]:
def session_last_click_aid(df_candidates, df_events):
    # 選取所有點擊事件
    df_clicks = df_events[df_events["type"] == "clicks"]
    
    # 按 session 排序並選擇每個 session 的最後一條點擊事件
    last_clicks = df_clicks.groupby("session").tail(1)[["session", "aid"]]
    
    # 重新命名列為 session_last_click_aid
    last_clicks = last_clicks.rename(columns={"aid": "session_last_click_aid"})
    
    # 透過 `.map()` 查找每個 session 的最後點擊商品 ID
    # 若該 session 沒有點擊事件，則設為 Nan
    last_click_aid = df_candidates["session"].map(last_clicks.set_index('session')['session_last_click_aid'])
    
    return last_click_aid

### session_last_atc_aid：最後一次加购的商品 ID

In [8]:
def session_last_atc_aid(df_candidates, df_events):
    # 選取所有加購 (加入購物車) 事件
    df_atc = df_events[df_events["type"] == "carts"]
    
    # 按 session 排序並選擇每個 session 的最後一條加購事件
    last_atc = df_atc.groupby("session").tail(1)[["session", "aid"]]
    
    # 重新命名列為 session_last_atc_aid
    last_atc = last_atc.rename(columns={"aid": "session_last_atc_aid"})
    
    # 透過 `.map()` 查找每個 session 的最後加購商品 ID
    # 若該 session 沒有加購事件，則設為 Nan
    last_atc_aid = df_candidates["session"].map(last_atc.set_index('session')['session_last_atc_aid'])
    
    return last_atc_aid

### session_last_order_aid：最後一次购买的商品 ID

In [9]:

def session_last_order_aid(df_candidates, df_events):
    # 選取所有購買 (buy) 事件
    df_orders = df_events[df_events["type"] == "orders"]
    
    # 按 session 排序並選擇每個 session 的最後一條購買事件
    last_order = df_orders.groupby("session").tail(1)[["session", "aid"]]
    
    # 重新命名列為 session_last_order_aid
    last_order = last_order.rename(columns={"aid": "session_last_order_aid"})
    
    # 透過 `.map()` 查找每個 session 的最後購買商品 ID
    # 若該 session 沒有購買事件，則設為 Nan
    last_order_aid = df_candidates["session"].map(last_order.set_index('session')['session_last_order_aid'])
    
    return last_order_aid

### session_last_click_timestamp：最後点击時間

In [10]:
def session_last_click_timestamp(df_candidates, df_events):
    # 選取所有點擊 (clicks) 事件
    df_clicks = df_events[df_events["type"] == "clicks"]
    
    # 取得每個 session 最後一次點擊的時間
    last_click_ts = df_clicks.groupby("session")["ts"].max().reset_index()
    
    # 重新命名欄位
    last_click_ts = last_click_ts.rename(columns={"ts": "session_last_click_timestamp"})
    
    # 透過 `.map()` 查找每個 session 的最後點擊時間
    last_click_timestamp = df_candidates["session"].map(last_click_ts.set_index("session")["session_last_click_timestamp"])
    
    # 若該 session 沒有點擊事件，則設為 Nan
    return last_click_timestamp

### session_last_atc_timestamp：最後加购時間

In [11]:
def session_last_atc_timestamp(df_candidates, df_events):
    # 選取所有加入購物車 (carts) 事件
    df_carts = df_events[df_events["type"] == "carts"]
    
    # 取得每個 session 最後一次加購的時間
    last_atc_ts = df_carts.groupby("session")["ts"].max().reset_index()
    
    # 重新命名欄位
    last_atc_ts = last_atc_ts.rename(columns={"ts": "session_last_atc_timestamp"})
    
    # 透過 `.map()` 查找每個 session 的最後加購時間
    last_atc_timestamp = df_candidates["session"].map(last_atc_ts.set_index("session")["session_last_atc_timestamp"])
    
    # 若該 session 沒有加購事件，則設為 Nan
    return last_atc_timestamp

### session_last_order_timestamp：最後购买時間

In [12]:
def session_last_order_timestamp(df_candidates, df_events):
    # 選取所有購買 (buys) 事件
    df_buys = df_events[df_events["type"] == "orders"]
    
    # 取得每個 session 最後一次購買的時間
    last_order_ts = df_buys.groupby("session")["ts"].max().reset_index()
    
    # 重新命名欄位
    last_order_ts = last_order_ts.rename(columns={"ts": "session_last_order_timestamp"})
    
    # 透過 `.map()` 查找每個 session 的最後購買時間
    last_order_timestamp = df_candidates["session"].map(last_order_ts.set_index("session")["session_last_order_timestamp"])
    
    # 若該 session 沒有購買事件，則設為 Nan
    return last_order_timestamp

# 商品特徵

### item_popularity：該商品被點擊的總次數

In [13]:
def item_popularity(df_candidates, df_events):
    # 計算每個商品被點擊的總次數
    click_counts = df_events[df_events["type"] == "clicks"].groupby("aid").size()
    
    # 將結果對應到候選集，沒有點擊記錄的商品填充 0
    return df_candidates["aid"].map(click_counts).fillna(0).astype(int).values

### item_cart_count：該商品被加入購物車的次數

In [14]:
def item_cart_count(df_candidates, df_events):
    # 計算每個商品被加入購物車的總次數
    cart_counts = df_events[df_events["type"] == "carts"].groupby("aid").size()
    
    # 將結果對應到候選集，沒有加入購物車記錄的商品填充 0
    return df_candidates["aid"].map(cart_counts).fillna(0).astype(int).values

### item_order_count：該商品的購買次數

In [15]:
def item_order_count(df_candidates, df_events):
    # 計算每個商品的購買總次數
    order_counts = df_events[df_events["type"] == "orders"].groupby("aid").size()
    
    # 將結果對應到候選集，沒有購買記錄的商品填充 0
    return df_candidates["aid"].map(order_counts).fillna(0).astype(int).values

### item_conversion_rate：購買次數 / 點擊次數

In [16]:
def item_conversion_rate(df_candidates, df_events):
    # 計算每個商品的購買次數
    order_counts = df_events[df_events["type"] == "orders"].groupby("aid").size()
    # 計算每個商品的點擊次數
    click_counts = df_events[df_events["type"] == "clicks"].groupby("aid").size()
    
    # 確保所有候選商品都有點擊和購買記錄，沒有則填充 0
    orders = df_candidates["aid"].map(order_counts).fillna(0)
    clicks = df_candidates["aid"].map(click_counts).fillna(0)
    
    # 計算轉換率，避免除以 0
    conversion_rate = (orders / clicks).fillna(0).round(4)
    
    return conversion_rate.values

### item_atc_rate：加购次數 / 點擊次數

In [17]:
def item_atc_rate(df_candidates, df_events):
    # 計算每個商品的加購次數
    atc_counts = df_events[df_events["type"] == "carts"].groupby("aid").size()
    # 計算每個商品的點擊次數
    click_counts = df_events[df_events["type"] == "clicks"].groupby("aid").size()
    
    # 確保所有候選商品都有點擊和加購記錄，沒有則填充 0
    atcs = df_candidates["aid"].map(atc_counts).fillna(0)
    clicks = df_candidates["aid"].map(click_counts).fillna(0)
    
    # 計算加購率，避免除以 0
    atc_rate = (atcs / clicks).fillna(0).round(4)
    
    return atc_rate.values

### item_atc_conversion_rate：购买次數 / 加购次數

In [18]:
def item_atc_conversion_rate(df_candidates, df_events):
    # 計算每個商品的購買次數
    order_counts = df_events[df_events["type"] == "orders"].groupby("aid").size()
    # 計算每個商品的加購次數
    atc_counts = df_events[df_events["type"] == "carts"].groupby("aid").size()
    
    # 確保所有候選商品都有加購和購買記錄，沒有則填充 0
    orders = df_candidates["aid"].map(order_counts).fillna(0)
    atcs = df_candidates["aid"].map(atc_counts).fillna(0)
    
    # 計算加購轉換率，避免除以 0
    atc_conversion_rate = (orders / atcs).fillna(0).round(4)
    
    return atc_conversion_rate.values

### item_recent_clicks：過去 7 天內該商品的點擊數

In [19]:
def item_recent_clicks(df_candidates, df_events_A):
    """
    計算item_recent_clicks特徵：events_A中最晚時間前推一週內每個候選商品的點擊數
    直接在df_candidates上進行merge操作
    
    Parameters:
    df_candidates (pd.DataFrame): 候選商品資料，包含 session, aid, score
    df_events_A (pd.DataFrame): A資料集的購買行為資料，包含 session, aid, ts, type
    
    Returns:
    np.array: 特徵陣列，長度等於 df_candidates 的行數
    """
    # 保存原始索引以便正確返回數組
    original_index = df_candidates.index.copy()
    
    # 1. 找出events_A中的最晚時間
    latest_ts = df_events_A['ts'].max()
    
    # 2. 計算一週前的時間戳（毫秒）
    one_week_before = latest_ts - (7 * 24 * 60 * 60 * 1000)
    
    # 3. 過濾出最近一週的點擊事件
    mask = (df_events_A['ts'] > one_week_before) & (df_events_A['type'] == 'clicks')
    
    # 4. 計算每個商品的點擊數
    aid_click_counts = df_events_A.loc[mask].groupby('aid').size().reset_index(name='click_count')
    
    # 5. 直接在df_candidates上進行merge操作
    result = df_candidates.merge(
        aid_click_counts,
        left_on='aid',
        right_on='aid',
        how='left'
    )['click_count'].fillna(0).astype(np.int32)
    
    # 確保結果陣列與輸入的df_candidates長度一致，並按原始順序排列
    # 由於merge可能會改變順序，我們需要確保按原始索引返回
    result = result.reindex(original_index).values
    
    return result

### item_recent_carts：過去 7 天內該商品的購物車數

In [20]:

def item_recent_carts(df_candidates, df_events_A):
    """
    計算item_recent_carts特徵：events_A中最晚時間前推一週內每個候選商品的加購次數
    直接在df_candidates上進行merge操作
    
    Parameters:
    df_candidates (pd.DataFrame): 候選商品資料，包含 session, aid, score
    df_events_A (pd.DataFrame): A資料集的購買行為資料，包含 session, aid, ts, type
    
    Returns:
    np.array: 特徵陣列，長度等於 df_candidates 的行數
    """
    # 保存原始索引以便正確返回數組
    original_index = df_candidates.index.copy()
    
    # 1. 找出events_A中的最晚時間
    latest_ts = df_events_A['ts'].max()
    
    # 2. 計算一週前的時間戳（毫秒）
    one_week_before = latest_ts - (7 * 24 * 60 * 60 * 1000)
    
    # 3. 過濾出最近一週的加入購物車事件
    mask = (df_events_A['ts'] > one_week_before) & (df_events_A['type'] == 'carts')
    
    # 4. 計算每個商品的加購次數
    aid_cart_counts = df_events_A.loc[mask].groupby('aid').size().reset_index(name='cart_count')
    
    # 5. 直接在df_candidates上進行merge操作
    result = df_candidates.merge(
        aid_cart_counts,
        left_on='aid',
        right_on='aid',
        how='left'
    )['cart_count'].fillna(0).astype(np.int32)
    
    # 確保結果陣列與輸入的df_candidates長度一致，並按原始順序排列
    # 由於merge可能會改變順序，我們需要確保按原始索引返回
    result = result.reindex(original_index).values
    
    return result

### item_recent_orders：過去 7 天內該商品的購買數

In [21]:
def item_recent_orders(df_candidates, df_events_A):
    """
    計算item_recent_orders特徵：events_A中最晚時間前推一週內每個候選商品的購買次數
    直接在df_candidates上進行merge操作
    
    Parameters:
    df_candidates (pd.DataFrame): 候選商品資料，包含 session, aid, score
    df_events_A (pd.DataFrame): A資料集的購買行為資料，包含 session, aid, ts, type
    
    Returns:
    np.array: 特徵陣列，長度等於 df_candidates 的行數
    """
    # 保存原始索引以便正確返回數組
    original_index = df_candidates.index.copy()
    
    # 1. 找出events_A中的最晚時間
    latest_ts = df_events_A['ts'].max()
    
    # 2. 計算一週前的時間戳（毫秒）
    one_week_before = latest_ts - (7 * 24 * 60 * 60 * 1000)
    
    # 3. 過濾出最近一週的購買事件
    mask = (df_events_A['ts'] > one_week_before) & (df_events_A['type'] == 'orders')
    
    # 4. 計算每個商品的購買次數
    aid_order_counts = df_events_A.loc[mask].groupby('aid').size().reset_index(name='order_count')
    
    # 5. 直接在df_candidates上進行merge操作
    result = df_candidates.merge(
        aid_order_counts,
        left_on='aid',
        right_on='aid',
        how='left'
    )['order_count'].fillna(0).astype(np.int32)
    
    # 確保結果陣列與輸入的df_candidates長度一致，並按原始順序排列
    # 由於merge可能會改變順序，我們需要確保按原始索引返回
    result = result.reindex(original_index).values
    
    return result

### item_type_mean：对不同种类的行为赋予数值编码后(click-0, atc-1, order-2)，计算所有行为的平均值。

In [22]:
def item_type_mean(df_candidates, df_events_A):
    """
    計算item_type_mean特徵：對不同種類的行為進行數值編碼後，計算所有行為的平均值
    行為類型：clicks -> 0, carts -> 1, buys -> 2
    
    Parameters:
    df_candidates (pd.DataFrame): 候選商品資料，包含 session, candidate_aid, score
    df_events_A (pd.DataFrame): A資料集的購買行為資料，包含 session, aid, ts, type
    
    Returns:
    np.array: 特徵陣列，長度等於 df_candidates 的行數
    """
    # 保存原始索引以便正確返回數組
    original_index = df_candidates.index.copy()
    
    # 1. 設定行為的數值編碼：clicks=0, carts=1, buys=2
    event_type_mapping = {
        'clicks': 0,
        'carts': 1,
        'orders': 2
    }
    
    # 2. 進行數值編碼
    df_events_A['event_type_encoded'] = df_events_A['type'].map(event_type_mapping)
    
    # 3. 計算每個商品的所有行為平均值
    item_type_mean = df_events_A.groupby('aid')['event_type_encoded'].mean().reset_index(name='item_type_mean')
    
    # 4. 直接在df_candidates上進行merge操作
    result = df_candidates.merge(
        item_type_mean,
        left_on='aid',
        right_on='aid',
        how='left'
    )['item_type_mean'].fillna(0).astype(np.float32)
    
    # 確保結果陣列與輸入的df_candidates長度一致，並按原始順序排列
    result = result.reindex(original_index).values
    
    return result

# 交互特徵

### session_item_click_count：該 session 內點擊該商品的次數

In [23]:
def session_item_click_count(df_candidates, df_events_B):
    """计算候选商品在其 session 内被点击的次数"""
    original_index = df_candidates.index.copy()

    df_clicks = df_events_B[df_events_B['type'] == 'clicks']
    click_counts = df_clicks.groupby(['session', 'aid']).size().reset_index(name='click_count')

    df_candidates_with_clicks = df_candidates.merge(
        click_counts, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    )

    result = df_candidates_with_clicks['click_count'].fillna(0).astype(np.int32)
    return result.reindex(original_index).values

### session_item_cart_count：該 session 內將該商品加入購物車的次數

In [24]:
def session_item_cart_count(df_candidates, df_events_B):
    """计算候选商品在其 session 内被加入购物车的次数"""
    original_index = df_candidates.index.copy()

    df_carts = df_events_B[df_events_B['type'] == 'carts']
    cart_counts = df_carts.groupby(['session', 'aid']).size().reset_index(name='cart_count')

    df_candidates_with_carts = df_candidates.merge(
        cart_counts, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    )

    result = df_candidates_with_carts['cart_count'].fillna(0).astype(np.int32)
    return result.reindex(original_index).values

### session_item_order_count：該 session 內將該商品購買的次數

In [25]:
def session_item_order_count(df_candidates, df_events_B):
    """计算候选商品在其 session 内被购买的次数"""
    original_index = df_candidates.index.copy()

    df_orders = df_events_B[df_events_B['type'] == 'orders']
    order_counts = df_orders.groupby(['session', 'aid']).size().reset_index(name='order_count')

    df_candidates_with_orders = df_candidates.merge(
        order_counts, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    )

    result = df_candidates_with_orders['order_count'].fillna(0).astype(np.int32)
    return result.reindex(original_index).values

### session_item_last_interaction_type：該商品所處 session 的最後一次事件类型，要get dummy

In [26]:
def session_item_last_interaction_type(df_candidates, df_events_B):
    """计算候选商品所处 session 的最后一次事件类型，并进行 One-Hot 编码"""
    original_index = df_candidates.index.copy()

    # 获取每个 session 的最后一次事件类型（由于数据已排序，直接使用 tail(1)）
    last_interaction_types = df_events_B.groupby('session').tail(1)[['session', 'type']]

    # 合并到 df_candidates
    df_candidates_with_type = df_candidates.merge(last_interaction_types, on='session', how='left')

    # 进行 One-Hot 编码
    interaction_dummies = pd.get_dummies(df_candidates_with_type['type'], prefix='last_interaction_type', drop_first=True)

    # 确保顺序一致
    interaction_dummies = interaction_dummies.reindex(original_index).fillna(0).astype(np.int32)

    return interaction_dummies.values

### session_item_abs_click_hots：用户点击该商品次数-该商品点击次数平均值

In [27]:
def session_item_abs_click_hots(df_candidates, df_events_A, df_events_B):
    """计算 session_item_abs_click_hots = 该商品在 session 内的点击次数 - 该商品在所有 session 的点击均值"""
    original_index = df_candidates.index.copy()

    # 1. 计算 用户点击该商品次数（某商品在其 session 内的点击次数）
    df_clicks_B = df_events_B[df_events_B['type'] == 'clicks']
    session_item_clicks = df_clicks_B.groupby(['session', 'aid']).size().reset_index(name='session_item_click_count')

    # 2. 计算 该商品在所有 session 的点击均值（某商品在 df_events_A 中的点击均值）
    df_clicks_A = df_events_A[df_events_A['type'] == 'clicks']
    item_click_avg = df_clicks_A.groupby('aid').size().div(
        df_clicks_A.groupby('aid')['session'].nunique()
    ).reset_index(name='item_click_avg')


    # 3. 合并 session 内点击次数到 df_candidates
    df_candidates_with_clicks = df_candidates.merge(
        session_item_clicks, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    ).merge(
        item_click_avg, left_on='aid', right_on='aid', how='left'
    )

    # 4. 计算 session_item_abs_click_hots
    df_candidates_with_clicks['session_item_abs_click_hots'] = (
        df_candidates_with_clicks['session_item_click_count'].fillna(0) - 
        df_candidates_with_clicks['item_click_avg'].fillna(0)
    )

    result = df_candidates_with_clicks['session_item_abs_click_hots'].astype(np.float32)
    return result.reindex(original_index).values

### session_item_abs_atc_hots：用户加购该商品次数-该商品加购次数平均值

In [28]:
def session_item_abs_atc_hots(df_candidates, df_events_A, df_events_B):
    """计算 session_item_abs_atc_hots = 该商品在 session 内的加购次数 - 该商品在所有 session 的加购均值"""
    original_index = df_candidates.index.copy()

    # 1. 计算 用户加购该商品次数（某商品在其 session 内的加购次数）
    df_carts_B = df_events_B[df_events_B['type'] == 'carts']
    session_item_carts = df_carts_B.groupby(['session', 'aid']).size().reset_index(name='session_item_cart_count')

    # 2. 计算 该商品在所有 session 的加购均值（某商品在 df_events_A 中的加购均值）
    df_carts_A = df_events_A[df_events_A['type'] == 'carts']
    item_cart_avg = df_carts_A.groupby('aid').size().div(
        df_carts_A.groupby('aid')['session'].nunique()
    ).reset_index(name='item_cart_avg')

    # 3. 合并 session 内加购次数到 df_candidates
    df_candidates_with_carts = df_candidates.merge(
        session_item_carts, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    ).merge(
        item_cart_avg, left_on='aid', right_on='aid', how='left'
    )

    # 4. 计算 session_item_abs_atc_hots
    df_candidates_with_carts['session_item_abs_atc_hots'] = (
        df_candidates_with_carts['session_item_cart_count'].fillna(0) - 
        df_candidates_with_carts['item_cart_avg'].fillna(0)
    )

    result = df_candidates_with_carts['session_item_abs_atc_hots'].astype(np.float32)
    return result.reindex(original_index).values

### session_item_abs_order_hots：用户购买该商品次数-该商品购买次数平均值

In [29]:
def session_item_abs_order_hots(df_candidates, df_events_A, df_events_B):
    """计算 session_item_abs_order_hots = 该商品在 session 内的购买次数 - 该商品在所有 session 的购买均值"""
    original_index = df_candidates.index.copy()

    # 1. 计算 用户购买该商品次数（某商品在其 session 内的购买次数）
    df_orders_B = df_events_B[df_events_B['type'] == 'orders']
    session_item_orders = df_orders_B.groupby(['session', 'aid']).size().reset_index(name='session_item_order_count')

    # 2. 计算 该商品在所有 session 的购买均值（某商品在 df_events_A 中的购买均值）
    df_orders_A = df_events_A[df_events_A['type'] == 'orders']
    item_order_avg = df_orders_A.groupby('aid').size().div(
        df_orders_A.groupby('aid')['session'].nunique()
    ).reset_index(name='item_order_avg')

    # 3. 合并 session 内购买次数到 df_candidates
    df_candidates_with_orders = df_candidates.merge(
        session_item_orders, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    ).merge(
        item_order_avg, left_on='aid', right_on='aid', how='left'
    )

    # 4. 计算 session_item_abs_order_hots
    df_candidates_with_orders['session_item_abs_order_hots'] = (
        df_candidates_with_orders['session_item_order_count'].fillna(0) - 
        df_candidates_with_orders['item_order_avg'].fillna(0)
    )

    result = df_candidates_with_orders['session_item_abs_order_hots'].astype(np.float32)
    return result.reindex(original_index).values

### session_item_abs_click_time：用户点击该商品时间-该商品点击时间平均值

In [30]:
def session_item_abs_click_time(df_candidates, df_events_A, df_events_B):
    """
    计算 session_item_abs_click_time：
    该商品在 session 内的平均点击时间 - 该商品在所有 session 内的平均点击时间。
    如果候选商品未在 session 内被点击，则返回 -1。

    参数:
    df_candidates (pd.DataFrame): 候选商品表，包含 session 和 aid。
    df_events_A (pd.DataFrame): A 数据集的点击行为表（全局数据）。
    df_events_B (pd.DataFrame): B 数据集的点击行为表（当前 session 的数据）。

    返回:
    np.array: 特征数组，长度等于 df_candidates 的行数。
    """
    original_index = df_candidates.index.copy()

    # 1计算 session 内该商品的平均点击时间
    df_clicks_B = df_events_B[df_events_B['type'] == 'clicks']
    session_item_click_time = df_clicks_B.groupby(['session', 'aid'])['ts'].mean().reset_index(name='session_item_click_time')

    # 计算该商品在所有 session 的平均点击时间
    df_clicks_A = df_events_A[df_events_A['type'] == 'clicks']
    item_click_time_avg = df_clicks_A.groupby('aid')['ts'].mean().reset_index(name='item_click_time_avg')

    # 合并 session 内点击时间到 df_candidates
    df_candidates_with_click_time = df_candidates.merge(
        session_item_click_time, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    ).merge(
        item_click_time_avg, left_on='aid', right_on='aid', how='left'
    )

    # 计算 session_item_abs_click_time
    df_candidates_with_click_time['session_item_abs_click_time'] = np.where(
        df_candidates_with_click_time['session_item_click_time'].notna(), 
        df_candidates_with_click_time['session_item_click_time'] - df_candidates_with_click_time['item_click_time_avg'], 
        np.nan
    )

    result = df_candidates_with_click_time['session_item_abs_click_time']
    return result.reindex(original_index).values


### session_item_abs_atc_time：用户加购该商品时间-该商品加购时间平均值

In [31]:
def session_item_abs_atc_time(df_candidates, df_events_A, df_events_B):
    """
    计算 session_item_abs_atc_time：
    该商品在 session 内的平均加购时间 - 该商品在所有 session 内的平均加购时间。
    如果候选商品未在 session 内被加购，则返回 -1。

    参数:
    df_candidates (pd.DataFrame): 候选商品表，包含 session 和 aid。
    df_events_A (pd.DataFrame): A 数据集的加购行为表（全局数据）。
    df_events_B (pd.DataFrame): B 数据集的加购行为表（当前 session 的数据）。

    返回:
    np.array: 特征数组，长度等于 df_candidates 的行数。
    """
    original_index = df_candidates.index.copy()

    # 计算 session 内该商品的平均加购时间
    df_carts_B = df_events_B[df_events_B['type'] == 'carts']
    session_item_cart_time = df_carts_B.groupby(['session', 'aid'])['ts'].mean().reset_index(name='session_item_cart_time')

    # 计算该商品在所有 session 的平均加购时间
    df_carts_A = df_events_A[df_events_A['type'] == 'carts']
    item_cart_time_avg = df_carts_A.groupby('aid')['ts'].mean().reset_index(name='item_cart_time_avg')

    # 合并 session 内加购时间到 df_candidates
    df_candidates_with_cart_time = df_candidates.merge(
        session_item_cart_time, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    ).merge(
        item_cart_time_avg, left_on='aid', right_on='aid', how='left'
    )

    # 计算 session_item_abs_atc_time
    df_candidates_with_cart_time['session_item_abs_atc_time'] = np.where(
        df_candidates_with_cart_time['session_item_cart_time'].notna(), 
        df_candidates_with_cart_time['session_item_cart_time'] - df_candidates_with_cart_time['item_cart_time_avg'], 
        np.nan
    )

    result = df_candidates_with_cart_time['session_item_abs_atc_time']
    return result.reindex(original_index).values


### session_item_abs_order_time：用户购买该商品时间-该商品购买时间平均值

In [32]:
def session_item_abs_order_time(df_candidates, df_events_A, df_events_B):
    """
    计算 session_item_abs_order_time:
    该商品在 session 内的平均购买时间 - 该商品在所有 session 内的平均购买时间。
    如果候选商品未在 session 内被购买，则返回 -1。

    参数:
    df_candidates (pd.DataFrame): 候选商品表，包含 session 和 candidate_aid。
    df_events_A (pd.DataFrame): A 数据集的购买行为表（全局数据）。
    df_events_B (pd.DataFrame): B 数据集的购买行为表（当前 session 的数据）。

    返回:
    np.array: 特征数组，长度等于 df_candidates 的行数。
    """
    original_index = df_candidates.index.copy()

    # 计算 session 内该商品的平均购买时间
    df_orders_B = df_events_B[df_events_B['type'] == 'order']
    session_item_order_time = df_orders_B.groupby(['session', 'aid'])['ts'].mean().reset_index(name='session_item_order_time')

    # 计算该商品在所有 session 的平均购买时间
    df_orders_A = df_events_A[df_events_A['type'] == 'order']
    item_order_time_avg = df_orders_A.groupby('aid')['ts'].mean().reset_index(name='item_order_time_avg')

    # 合并 session 内购买时间到 df_candidates
    df_candidates_with_order_time = df_candidates.merge(
        session_item_order_time, left_on=['session', 'aid'], right_on=['session', 'aid'], how='left'
    ).merge(
        item_order_time_avg, left_on='aid', right_on='aid', how='left'
    )

    # 计算 session_item_abs_order_time
    df_candidates_with_order_time['session_item_abs_order_time'] = np.where(
        df_candidates_with_order_time['session_item_order_time'].notna(), 
        df_candidates_with_order_time['session_item_order_time'] - df_candidates_with_order_time['item_order_time_avg'], 
        np.nan
    )

    result = df_candidates_with_order_time['session_item_abs_order_time']
    return result.reindex(original_index).values

# 整合

In [33]:
def save_features_to_parquet(df_candidates, df_events_A, df_events_B):
    # 建立 DataFrame，以 session 為索引
    df_features = pd.DataFrame({"session": df_candidates["session"], "aid": df_candidates["aid"]})

    # 計算session特徵
    df_features["session_length"] = session_length(df_candidates, df_events_B)
    df_features["session_click_ratio"] = session_click_ratio(df_candidates, df_events_B)
    df_features["session_cart_ratio"] = session_cart_ratio(df_candidates, df_events_B)
    df_features["session_order_ratio"] = session_order_ratio(df_candidates, df_events_B)
    df_features["session_last_click_aid"] = session_last_click_aid(df_candidates, df_events_B)
    df_features["session_last_atc_aid"] = session_last_atc_aid(df_candidates, df_events_B)
    df_features["session_last_order_aid"] = session_last_order_aid(df_candidates, df_events_B)
    df_features["session_last_click_timestamp"] = session_last_click_timestamp(df_candidates, df_events_B)
    df_features["session_last_atc_timestamp"] = session_last_atc_timestamp(df_candidates, df_events_B)
    df_features["session_last_order_timestamp"] = session_last_order_timestamp(df_candidates, df_events_B)

    # 計算商品特徵
    df_features["item_popularity"] = item_popularity(df_candidates, df_events_A)
    df_features["item_cart_count"] = item_cart_count(df_candidates, df_events_A)
    df_features["item_order_count"] = item_order_count(df_candidates, df_events_A)
    df_features["item_conversion_rate"] = item_conversion_rate(df_candidates, df_events_A)
    df_features["item_atc_rate"] = item_atc_rate(df_candidates, df_events_A)
    df_features["item_atc_conversion_rate"] = item_atc_conversion_rate(df_candidates, df_events_A)
    df_features["item_recent_clicks"] = item_recent_clicks(df_candidates, df_events_A)
    df_features["item_recent_carts"] = item_recent_carts(df_candidates, df_events_A)
    df_features["item_recent_orders"] = item_recent_orders(df_candidates, df_events_A)
    df_features["item_type_mean"] = item_type_mean(df_candidates, df_events_A)

    #計算交互特徵
    df_features["session_item_click_count"] = session_item_click_count(df_candidates, df_events_B)
    df_features["session_item_cart_count"] = session_item_cart_count(df_candidates, df_events_B)
    df_features["session_item_order_count"] = session_item_order_count(df_candidates, df_events_B)
    df_features["session_item_last_interaction_type_clicks"] = session_item_last_interaction_type(df_candidates, df_events_B)[:,0]
    df_features["session_item_last_interaction_type_orders"] = session_item_last_interaction_type(df_candidates, df_events_B)[:,1]
    df_features["session_item_abs_click_hots"] = session_item_abs_click_hots(df_candidates, df_events_A, df_events_B)
    df_features["session_item_abs_atc_hots"] = session_item_abs_atc_hots(df_candidates, df_events_A, df_events_B)
    df_features["session_item_abs_order_hots"] = session_item_abs_order_hots(df_candidates, df_events_A, df_events_B)
    df_features["session_item_abs_click_time"] = session_item_abs_click_time(df_candidates, df_events_A, df_events_B)
    df_features["session_item_abs_atc_time"] = session_item_abs_atc_time(df_candidates, df_events_A, df_events_B)
    df_features["session_item_abs_order_time"] = session_item_abs_order_time(df_candidates, df_events_A, df_events_B)

    # 存成 parquet
    # df_features.to_csv("session_features_sessionXaid.csv", index=False)
    # df_features.to_parquet("session_features_sessionXaid.parquet", index=False)
    df_features.to_parquet("data/2_session_features_sessionXaid.parquet", index=False)

    print("Parquet saved successfully")

In [34]:
file_path = "data/train_week1-3/"

file_list = [f"{file_path}{str(i).zfill(3)}.parquet" for i in range(111)]

df_list = [pd.read_parquet(file) for file in file_list]
df_events_A = pd.concat(df_list, ignore_index=True)

print(f"合并后的 DataFrame 形状: {df_events_A.shape}")
print(df_events_A.head())

合并后的 DataFrame 形状: (163955180, 4)
   session      aid             ts    type
0        0  1517085  1659304800025  clicks
1        0  1563459  1659304904511  clicks
2        0  1309446  1659367439426  clicks
3        0    16246  1659367719997  clicks
4        0  1781822  1659367871344  clicks


In [35]:
file_path = "data/val_partA/"

file_list = [f"{file_path}{str(i).zfill(3)}.parquet" for i in range(19)]

df_list = [pd.read_parquet(file) for file in file_list]
df_events_B = pd.concat(df_list, ignore_index=True)

print(f"合并后的 DataFrame 形状: {df_events_B.shape}")
print(df_events_B.head())

合并后的 DataFrame 形状: (7683577, 4)
    session      aid             ts    type
0  11098528    11830  1661119200060  clicks
1  11098529  1105029  1661119200259  clicks
2  11098530   264500  1661119200974  clicks
3  11098530   264500  1661119288407  clicks
4  11098530   409236  1661119369986  clicks


In [36]:
df_candidates = pd.read_parquet("data/candidates.parquet")
print(df_candidates.shape)
print(df_candidates.head())

(90062550, 2)
    session      aid
0  11098528    11830
1  11098528   588923
2  11098528  1732105
3  11098528   571762
4  11098528   884502


In [37]:
save_features_to_parquet(df_candidates, df_events_A, df_events_B)

Parquet saved successfully
